In [1]:
import pandas as pd
import wrangle
import model

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from statsmodels.formula.api import ols
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.feature_selection import f_regression, SelectKBest, RFE 
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
from math import sqrt
from scipy import stats

In [2]:
df, train_exp, X_train_scaled, y_train, X_test_scaled, y_test = wrangle.wrangle_data()

Acquire: compiling raw data files...
Acquire: Completed!
Prepare: preparing data files...
Prepare: Completed!
(289, 25) (73, 25)


# Create Baseline

In [3]:
baseRMSE, basepred = model.get_baseline(y_train)

Baseline RMSE: 1286.801141497012


# Feature Ranking

In [6]:
# possible feature ranking?
rankdf = model.feature_ranking(X_train_scaled, y_train)
rankdf

,features
rank,
1,rank_svi_scaled
2,f_nohsdp_soci
3,f_minrty_status
4,f_groupq_trans
5,f_comp_total_scaled
6,f_unemp_soci
7,f_disabl_comp
8,f_noveh_trans
9,f_mobile_trans


# Create X_train_scaled df with selected features to test

## Feature groups to use for OLS, PF, and Tweedie models

In [ ]:
# binned svi score by CDC range category = 1st ranked
X_rank_svi_only = X_train_scaled[['rank_svi_scaled']]
# top 4 ranked features
X_top5 = X_train_scaled[['rank_svi_scaled', 'f_nohsdp_soci', 'f_minrty_status', 'f_groupq_trans']]
# only the summary of the flags = 19th ranked
X_all_flags_only = X_train_scaled[['all_flags_total_scaled']]
# only summary flags, should be the same as all flags total? = 5th, 12th, 15th, 21st
X_summary_flags = X_train_scaled[['f_comp_total_scaled', 'f_soci_total_scaled', 'f_status_total_scaled', 'f_trans_total_scaled']]
# all individual flags
X_not_summary_flags = X_train_scaled[['f_nohsdp_soci', 'f_minrty_status', 'f_groupq_trans', 'f_unemp_soci', 
                                     'f_disabl_comp', 'f_noveh_trans', 'f_mobile_trans', 'f_age65_comp', 
                                     'f_age17_comp', 'f_pov_soci', 'f_limeng_status', 'f_crowd_trans', 'f_pci_soci' 
                                     'f_sngpnt_comp', 'f_munit_trans']]

## LassoLars
- includes feature selection as part of model so will use all features on this model

## Run all features on other models as well? -- RETURN HERE

## Drilling in features to test

- what is the score using only 1 summary flag at a time?
    - if one summary group is better might be worth investigating for which features within group have greatest impact?

# Run df through regression algorithms

In [ ]:
# create variables for loop
df2test = [Xtrains_ALL, Xtrains_noClusters, Xtrains_only_county, Xtrains_onlyLoc, Xtrains_onlyCentroids, 
           Xtrains_countyDollar, Xtrains_countySize, Xtrains_countyRooms]
target = y_train

# Linear Regression Models
lm_list = []
pred_list = []
RMSE_list = []
for df in df2test:
    lm, pred, RMSE = model.linear_reg_train(df, target)
    
    pred_list.append(pred)
    RMSE_list.append(RMSE)

# LassoLars Models
# this produces same result for all dataframes because this model does feature selection within the model and 
# is selecting the same features from all dataframes
for df in df2test:
    ll_RMSE = model.lasso_lars(df, target)
    
# Tweedie
tweedie = []
for df in df2test:
    tw_RMSE = model.tweedie(df, target)
    tweedie.append(tw_RMSE)

# adjusted the hyperparameter of alpha from .1 to 1 with worse result
# adjusted again to alpha = .001 for these better results

# Summarize in results df

In [ ]:
# create dataframe for results of all train models
df_list = ['all', 'no clusters', 'only county', 'only location', 'only centriods',
           'county with dollar', 'county with size', 'county with rooms']

results = pd.DataFrame(df_list, columns=['Features'])
results['LinearReg'] = RMSE_list
results['Tweedie'] = tweedie
results['Baseline'] = baseline
results.sort_values('LinearReg')